# Импорт данных

In [28]:
import os
import pandas as pd

file_path1 = "dataset1.csv"
file_path2 = "dataset2.csv"
file_path3 = "dataset3.xlsx"

## Read csv

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

Импортируем CSV dataset1.csv.
Кодировка utf-8.

Визуальный анализ всё же нужен. Поэтому представить себе, что можно неглядя импортировать данные - сложно. Лучше визуализировать сырые данные в табличных программах типа excel / numbers или в крайнем случае текстовый редактор для csv.

В случае с csv для начала нам нужно визуально в каком-либо редакторе определить разделитель значений <code>delimiter</code> и разделитель десятичных знаков у чисел <code>decimal</code>.

In [33]:
df1 = pd.read_csv(file_path1, delimiter = ";", decimal=",")
print(df1)
print("---------------")
print(df1.head(3))  # первые 3 строки
print("---------------")
print(df1.tail(4))  # последние 4 строки (можно вариировать для наглядности)
print("---------------")
print(df1.info())
# Лучше лишний раз использовать tail для просмотра окончания талицы, чем удалить нужные данные!!!

  Unnamed: 0    Unnamed: 1    Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0      INDEX  STRING_FIELD  NUMBER_FIELD  DATE_FIELD         NaN         NaN   
1        NaN           NaN           NaN         NaN         NaN         NaN   
2          1      Джигурда          1500  14.11.2020         NaN         NaN   
3          2    Э. Суровый       2000.15  21.05.2023         NaN         NaN   
4          3    Snoop Dogg   1000000,001  31.01.2022         NaN         NaN   
5          4      Галустян           1,5  01.01.2022         NaN         NaN   
6          5    Bill Gates   -523588,125  12.12.2020         NaN         NaN   
7       авыф        Итого:    479913,376         NaN         NaN         NaN   
8       фыва           NaN           NaN         NaN         NaN         NaN   
9        NaN           NaN           NaN       текст         NaN         NaN   

   Unnamed: 6  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
5         NaN  
6      

Данные могут быть не совсем структурированы и сырой импорт может дать печальный результат. Первая строка очевидно пуста (судя по отсутствующим названиям заголовков), заголовки находятся на 2 строке. Однако треться строка (сразу после заголовков) тоже пуста. В конце импортированной талицы тоже лишние строки - их номера в крайней левой позиции вывода.

Можно переимпортировать данные более избирательно. Для этого используем параметры <code>usecols</code> и <code>skiprows</code>. Для обоих в качестве значений используются __последовательности__ - листы, картежи или функции, их возвращающие. При выборе номеров строк для пропуска помним, что верхняя строка импортируется как заголовок и дальше список начинается с 0.

__Лучше лишний раз использовать tail для просмотра окончания талицы, чем удалить нужные данные!!!__

In [30]:
df1 = pd.read_csv(file_path1, delimiter = ";", decimal=",", usecols=range(4), skiprows=[0,2,8,9,10])
print(df1)
print("---------------")
print(df1.head(3))
print("---------------")
print(df1.tail(3))
print("---------------")
print(df1.info())

   INDEX STRING_FIELD NUMBER_FIELD  DATE_FIELD
0      1     Джигурда         1500  14.11.2020
1      2   Э. Суровый      2000.15  21.05.2023
2      3   Snoop Dogg  1000000,001  31.01.2022
3      4     Галустян          1,5  01.01.2022
4      5   Bill Gates  -523588,125  12.12.2020
---------------
   INDEX STRING_FIELD NUMBER_FIELD  DATE_FIELD
0      1     Джигурда         1500  14.11.2020
1      2   Э. Суровый      2000.15  21.05.2023
2      3   Snoop Dogg  1000000,001  31.01.2022
---------------
   INDEX STRING_FIELD NUMBER_FIELD  DATE_FIELD
2      3   Snoop Dogg  1000000,001  31.01.2022
3      4     Галустян          1,5  01.01.2022
4      5   Bill Gates  -523588,125  12.12.2020
---------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   INDEX         5 non-null      int64 
 1   STRING_FIELD  5 non-null      object
 2   NUMBER_FIELD  5 non-n

Дата в DATE_FIELD конвертировалась как строка. Перенесём её а тип данных для дат.

In [31]:
df1["DATE_FIELD"] = pd.to_datetime(df1["DATE_FIELD"], format="%d.%m.%Y")
print(df1)
print(df1.info())

   INDEX STRING_FIELD NUMBER_FIELD DATE_FIELD
0      1     Джигурда         1500 2020-11-14
1      2   Э. Суровый      2000.15 2023-05-21
2      3   Snoop Dogg  1000000,001 2022-01-31
3      4     Галустян          1,5 2022-01-01
4      5   Bill Gates  -523588,125 2020-12-12
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   INDEX         5 non-null      int64         
 1   STRING_FIELD  5 non-null      object        
 2   NUMBER_FIELD  5 non-null      object        
 3   DATE_FIELD    5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 288.0+ bytes
None


Поле NUMBER_FIELD должно быть числовым. Оно конвертировалось в строку из-за того, что в одном из числовых значений использован разделитель десятичных знаков ".", хотя мы указали ",".

Эту проблему можно решить сперва заменив все запятые на точки, а затем конвертировать поле. Для конвертации используется функция <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html" target=_blanc>to_numeric</a>, а для её корректной работы разделитель в исходных текстовх значениях должен быть ".".

In [32]:
df1["NUMBER_FIELD"] = pd.to_numeric(df1["NUMBER_FIELD"].str.replace(",", "."))
print(df1)
print(df1.info())

   INDEX STRING_FIELD  NUMBER_FIELD DATE_FIELD
0      1     Джигурда      1500.000 2020-11-14
1      2   Э. Суровый      2000.150 2023-05-21
2      3   Snoop Dogg   1000000.001 2022-01-31
3      4     Галустян         1.500 2022-01-01
4      5   Bill Gates   -523588.125 2020-12-12
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   INDEX         5 non-null      int64         
 1   STRING_FIELD  5 non-null      object        
 2   NUMBER_FIELD  5 non-null      float64       
 3   DATE_FIELD    5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 288.0+ bytes
None


Теперь данные готовы для использования.

Сделаем то же самое для excel файла dataset3.xlsx